<img src="https://68.media.tumblr.com/79dbb18434f983637e706857067b2fab/tumblr_nzeqww3Lep1tcvh9jo1_400.jpg"/>

# Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams

from preprocessing import *
from math_utils import *
from plotting import *

%matplotlib inline

sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 12, 6

IMAGES_PATH = 'train-images-idx3-ubyte'
LABELS_PATH = 'train-labels-idx1-ubyte'
RANDOM_SEED = 42

N_FEATURES = 28 * 28
N_CLASSES = 10

np.random.seed(RANDOM_SEED)

# Preliminaries

## The sigmoid function (and it's derivative)

In [ ]:
x = np.linspace(-10., 10., num=100)
sig = sigmoid(x)
sig_prime = sigmoid_prime(x)

plt.plot(x, sig, label="sigmoid")
plt.plot(x, sig_prime, label="sigmoid prime")
plt.xlabel("x")
plt.ylabel("y")
plt.legend(prop={'size' : 16})
plt.show()

## Softmax

The softmax function can be easily differentiated, it is pure (output depends only on input) and the elements of the resulting vector sum to 1:

In [ ]:
softmax(np.array([[2, 4, 6, 8]]))

## NumPy

Let's define 2 matrices:

In [ ]:
m1 = np.array([[1, 2, 3], [2, 3, 4]])
m2 = np.array([[3, 2, 1], [4, 3, 2]])

m1

In [ ]:
m1.shape

Transpose:

In [ ]:
m2.T

Scalar product:

In [ ]:
m1.dot(m2.T)

Matrix multiplication:

In [ ]:
np.multiply(m1, m2)

## Backpropagation

The algorithm consists of 3 subtasks:

* Make a forward pass
* Calculate the error
* Make backward pass (backpropagation)

In the first step, backprop uses the data and the weights of the network to compute a prediction. Next, the error is computed based on the prediction and the provided labels. The final step propagates the error through the network, starting from the final layer. Thus, the weights get updated based on the error, little by little.

We will try to create a Neural Network (NN) that can properly predict values from the *XOR* function:

| Input 1 	| Input 2 	| Output 	|
|---------	|---------	|--------	|
| 0       	| 0       	| 0      	|
| 0       	| 1       	| 1      	|
| 1       	| 0       	| 1      	|
| 1       	| 1       	| 0      	|

*XOR* can be plotted as:

<img src="https://wantee.github.io/assets/images/posts/xor.png" width="60%">

Let start by defining some parameters:

In [ ]:
epochs = 50000
input_size, hidden_size, output_size = 2, 3, 1
LR = .1 # learning rate

Our data looks like this:

In [ ]:
X = np.array([[0,0], [0,1], [1,0], [1,1]])
y = np.array([ [0],   [1],   [1],   [0]])

Initialize the weights of our NN to random numbers:

In [ ]:
w_hidden = np.random.uniform(size=(input_size, hidden_size))
w_output = np.random.uniform(size=(hidden_size, output_size))

Finally, implementation of the Backprop algorithm:

In [ ]:
for epoch in range(epochs):
 
    # Forward
    act_hidden = sigmoid(np.dot(X, w_hidden))
    output = np.dot(act_hidden, w_output)
    
    # Calculate error
    error = y - output
    
    if epoch % 5000 == 0:
        print(f'error sum {sum(error)}')

    # Backward
    dZ = error * LR
    w_output += act_hidden.T.dot(dZ)
    dH = dZ.dot(w_output.T) * sigmoid_prime(act_hidden)
    w_hidden += X.T.dot(dH)

Let's try to predict using our trained model (doing just the forward step):

In [ ]:
X_test = X[1] # [0, 1]

act_hidden = sigmoid(np.dot(X_test, w_hidden))
np.dot(act_hidden, w_output)

[Why NNs use biases?](https://stackoverflow.com/questions/2480650/role-of-bias-in-neural-networks)

# Splitting the data

In [ ]:
X, y = read_mnist(IMAGES_PATH, LABELS_PATH)
X, y = shuffle_data(X, y, random_seed=RANDOM_SEED)
X_train, y_train = X[:500], y[:500]
X_test, y_test = X[500:], y[500:]

# Data exploration

In [ ]:
plot_digit(X, y, idx=1)

# Building our model

In [ ]:
class NNClassifier:

    def __init__(self, n_classes, n_features, n_hidden_units=30,
                 l1=0.0, l2=0.0, epochs=500, learning_rate=0.01,
                 n_batches=1, random_seed=None):

        if random_seed:
            np.random.seed(random_seed)
        self.n_classes = n_classes
        self.n_features = n_features
        self.n_hidden_units = n_hidden_units
        self.w1, self.w2 = self._init_weights()
        self.l1 = l1
        self.l2 = l2
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.n_batches = n_batches

    def _init_weights(self):
        w1 = np.random.uniform(-1.0, 1.0, 
                               size=self.n_hidden_units * (self.n_features + 1))
        w1 = w1.reshape(self.n_hidden_units, self.n_features + 1)
        w2 = np.random.uniform(-1.0, 1.0, 
                               size=self.n_classes * (self.n_hidden_units + 1))
        w2 = w2.reshape(self.n_classes, self.n_hidden_units + 1)
        return w1, w2

    def _add_bias_unit(self, X, how='column'):
        """Add bias unit (column or row of 1s) to array at index 0"""
        if how == 'column':
            X_new = np.ones((X.shape[0], X.shape[1]+1))
            X_new[:, 1:] = X
        elif how == 'row':
            X_new = np.ones((X.shape[0]+1, X.shape[1]))
            X_new[1:, :] = X
        else:
            raise AttributeError('how must be columns or row')
        return X_new

    def _forward(self, X):
        raise NotImplementedError
    
    def _backward(self, net_input, net_hidden, act_hidden, act_out, y):
        raise NotImplementedError

    def _error(self, y, output):
        raise NotImplementedError

    def _backprop_step(self, X, y):
        net_input, net_hidden, act_hidden, net_out, act_out = self._forward(X)
        y = y.T

        grad1, grad2 = self._backward(net_input, net_hidden, act_hidden, act_out, y)

        # regularize
        grad1[:, 1:] += (self.w1[:, 1:] * (self.l1 + self.l2))
        grad2[:, 1:] += (self.w2[:, 1:] * (self.l1 + self.l2))

        error = self._error(y, act_out)
        
        return error, grad1, grad2

    def predict(self, X):
        raise NotImplementedError
    
    def predict_proba(self, X):
        raise NotImplementedError

    def fit(self, X, y):
        self.error_ = []
        X_data, y_data = X.copy(), y.copy()
        y_data_enc = one_hot(y_data, self.n_classes)
        for i in range(self.epochs):

            X_mb = np.array_split(X_data, self.n_batches)
            y_mb = np.array_split(y_data_enc, self.n_batches)

            for Xi, yi in zip(X_mb, y_mb):
                
                # update weights
                error, grad1, grad2 = self._backprop_step(Xi, yi)
                self.error_.append(error)
                self.w1 -= (self.learning_rate * grad1)
                self.w2 -= (self.learning_rate * grad2)

        return self
    
    def score(self, X, y):
        y_hat = self.predict(X)
        return np.sum(y == y_hat, axis=0) / float(X.shape[0])

## Training

In [ ]:
nn = NNClassifier(n_classes=N_CLASSES, 
                  n_features=N_FEATURES,
                  n_hidden_units=50,
                  l2=0.5,
                  l1=0.0,
                  epochs=300,
                  learning_rate=0.001,
                  n_batches=25,
                  random_seed=RANDOM_SEED)

nn.fit(X_train, y_train);

# Evaluation

In [ ]:
plot_error(nn)

In [ ]:
print('Train Accuracy: %.2f%%' % (nn.score(X_train, y_train) * 100))
print('Test Accuracy: %.2f%%' % (nn.score(X_test, y_test) * 100))

In [ ]:
nn.predict_proba(X_test[1:2])

## Correct prediction

In [ ]:
plot_digit(X_test, y_test, idx=1)

In [ ]:
plot_digit_dist(X_test, y_test, idx=1, model=nn)

## Wrong prediction

In [ ]:
plot_digit(X_test, y_test, idx=25)

In [ ]:
plot_digit_dist(X_test, y_test, idx=25, model=nn)

## MLE = picking the most probable digit

In [ ]:
nn.predict_proba(X_test[:5])

In [ ]:
mle(nn.predict_proba(X_test[:5]))

In [ ]:
nn.predict(X_test[:5])